**Image Classification for Crop**


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import tensorflow.keras.layers as Layers
import tensorflow.keras.activations as Actications
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer
import tensorflow.keras.metrics as Metrics
import tensorflow.keras.utils as Utils
from keras.utils.vis_utils import model_to_dot
import os
import matplotlib.pyplot as plot
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix as CM
from random import randint
from IPython.display import SVG
from keras.models import model_from_json

Loading Image Dataset


In [3]:
def loadImage(directory):
  Images = []
  Labels = [] #0=> Rice, 1=> Tomato, 2=> Potato, 3=> lady_finger, 4=> strawberry
  label = 0
  for nameDir in os.listdir(directory):
    if nameDir == 'rice':
      label = 0
    elif nameDir == 'tomato':
      label = 1
    elif nameDir == 'potato':
      label = 2
    elif nameDir == 'lady_finger':
      label = 3
    elif nameDir == 'strawberry':
      label = 4

    filePath = directory + nameDir

    for imageFile in os.listdir(filePath):
      image = cv2.imread(filePath + r'/' + imageFile)
      image = cv2.resize(image,(150,150))
      Images.append(image)
      Labels.append(label)

  return shuffle(Images,Labels,random_state = 8173)

  

In [4]:
directory = '/content/drive/MyDrive/dataset/'
Images,Labels = loadImage(directory)
Images = np.array(Images)
Labels = np.array(Labels)

Model Design and Save Model

In [5]:
totalClass = 5
model = Models.Sequential()

model.add(Layers.Conv2D(200,kernel_size=(3,3),activation='relu',input_shape=(150,150,3)))
model.add(Layers.Conv2D(180,kernel_size=(3,3),activation='relu'))
model.add(Layers.MaxPool2D(5,5))
model.add(Layers.Conv2D(180,kernel_size=(3,3),activation='relu'))
model.add(Layers.Conv2D(140,kernel_size=(3,3),activation='relu'))
model.add(Layers.Conv2D(100,kernel_size=(3,3),activation='relu'))
model.add(Layers.Conv2D(50,kernel_size=(3,3),activation='relu'))
model.add(Layers.MaxPool2D(5,5))
model.add(Layers.Flatten())
model.add(Layers.Dense(180,activation='relu'))
model.add(Layers.Dense(100,activation='relu'))
model.add(Layers.Dense(50,activation='relu'))
model.add(Layers.Dropout(rate=0.5))
model.add(Layers.Dense(totalClass,activation='softmax'))

model.compile(optimizer=Optimizer.Adam(lr=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

trained = model.fit(Images,Labels,epochs=25,validation_split=0.30)
model_json = model.to_json()
with open("/content/demo.json","w") as json_file:
	json_file.write(model_json)
model.save_weights("/content/model.h5")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 200)     5600      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 146, 180)     324180    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 180)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 180)       291780    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 140)       226940    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 23, 23, 100)       126100    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 21, 21, 50)        4

Test Model


In [8]:
img=cv2.imread("/content/drive/MyDrive/dataset/tomato/10.jpg")
img=cv2.resize(img,(150,150))
model.predict(np.array([img]))

array([[9.5969161e-05, 9.9356467e-01, 1.1571818e-04, 6.2605330e-05,
        6.1609810e-03]], dtype=float32)